Aplicando restrições na busca
=============================



## Introdução



Muitos problemas de otimização com relevância científica têm uma ou mais `restrições` que devem ser levadas em consideração na hora de resolver o problema.

Lembra do `problema da mochila` que vimos em Lógica Computacional? Era um problema de otimização onde queríamos maximizar o valor dos itens colocados na mochila enquanto observávamos a restrição do peso total dos itens (do contrário, a mochila rasgava).

Uma forma de considerar essas restrições nos problemas é aplicando uma `penalidade` na função objetivo.

Vamos pensar como seria essa penalidade no problema da mochila: a função objetivo é maximizar o valor dos itens na mochila, então é um problema de maximização. A função objetivo pode ser a soma dos itens da mochila. Se fosse só isso, teríamos

$$
f = \sum_{i, i \in \mathrm{mochila}}\mathrm{valor}(i)
$$

No entanto, apenas essa função não resolve o problema! Precisamos levar em consideração o limite de peso da mochila! Para isso, penalizamos a função objetivo levando em consideração essa restrição:

$f=\begin{cases}
0.01 & \textrm{se peso > limite da mochila}\\
\sum_{i,i\in\mathrm{mochila}}(\mathrm{valor}(i)) & \textrm{se peso} \leq \textrm{limite da mochila}
\end{cases}$

Agora finalmente podemos seguir em frente e resolver o problema.



## Reflexões



Se usarmos a equação de $f$ acima, qual será o valor de $f$ caso não exista uma solução para um certo problema da mochila?

Na equação de $f$ acima nós usamos um valor práximo de zero para indicar que uma restrição do problema não foi satisfeita. Você consegue pensar em outra estratégia para penalizar soluções inválidas?



## Objetivo



Encontrar uma solução para o problema da mochila usando algoritmos genéticos. Considere que existem 10 itens diferentes (com pesos e valores diferentes) disponíveis para serem escolhidos.



## Descrição do problema



No problema da mochila você tem um número $n$ de itens disponíveis, cada um com um peso e um valor associado. Sua mochila tem a capacidade de carregar um número $p$ de quilogramas, sendo que mais que isso faz com que sua mochila rasgue e todos os itens dentro dela caiam no chão e se quebrem de maneira catastrófica (indesejado). Sua tarefa é encontrar um conjunto de itens (considerando os $n$ disponíveis) que maximize o valor contido dentro da mochila, porém que tenham um peso dentro da capacidade da mesma.



## Importações



In [1]:
import time
import random
import itertools

from funcoes import computa_mochila
from funcoes import funcao_objetivo_pop_mochila
from funcoes import populacao_cb as cria_populacao_inicial
from funcoes import selecao_roleta_max as funcao_selecao
from funcoes import cruzamento_ponto_simples as funcao_cruzamento
from funcoes import mutacao_cb as funcao_mutacao

## Códigos e discussão



In [2]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [3]:
# Funções locais

def funcao_objetivo_pop(populacao):
    return funcao_objetivo_pop_mochila(
        populacao, OBJETOS, LIMITE_DE_PESO, ORDEM_DOS_NOMES
    )

In [4]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

inicio = time.time()

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")

fim = time.time()

# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

print(f'O código levou {round(fim-inicio,4)} segundos para ser executado')

Maior valor: 15500 | Peso: 13.5
Maior valor: 16550 | Peso: 14.1
Maior valor: 17900 | Peso: 14.1

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ tablet
+ teclado musical
+ vinil falsificado da volta do One Direction

Com isso, sua mochila terá o valor de 17900 dinheiros e peso de 14.1 unidades de massa.
O código levou 0.011 segundos para ser executado


#

**Desafio**: resolva o experimento considerando uma busca em grade para encontrar a melhor resposta.

In [5]:
melhor_valor, melhor_peso = float('-inf'), float('inf')

inicio = time.time()

for individuo in itertools.product([0,1], repeat=len(OBJETOS.keys())):
    valor,peso = computa_mochila(individuo,OBJETOS,ORDEM_DOS_NOMES)
    
    if peso <= LIMITE_DE_PESO:
        if valor > melhor_valor:
            melhor_valor = valor
            melhor_peso = peso

fim = time.time()

print(f"A melhor mochila possível tem {melhor_valor} dinheiros e peso de {melhor_peso} unidades de massa.")
print(f'O código levou {round(fim-inicio,4)} segundos para ser executado')

A melhor mochila possível tem 17900 dinheiros e peso de 14.1 unidades de massa.
O código levou 0.001 segundos para ser executado


#

**Testando com uma maior quantidade de elementos**

O tempo necessário da busca em grade foi menor do que o tempo necessário para o algoritmo genético. Por que isso pode ter acontecido? Propõe-se uma busca e aplicação dos algoritmos com uma variação da quantidade de objetos.

In [6]:
### CONSTANTES

# relacionadas à busca
TAMANHO_POP = 20
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.05

# relacionadas ao problema a ser resolvido
LIMITE_DE_PESO = 15
OBJETOS = {
    # dicionário baseado no que vocês enviaram na aula de Lógica

    "vinil falsificado da volta do One Direction": {
        "peso": 2,
        "valor": 2500,
    },
    "Harry Potter: ele voltou, confia!": {
        "peso": 3,
        "valor": 1500,
    },
    "Quadrinho super raro do Aranha-Homem da vida real": {
        "peso": 3,
        "valor": 7000,
    },
    "mesa dobrável para laptop": {
        "peso": 3,
        "valor": 150,
    },
    "tablet": {
        "peso": 0.6,
        "valor": 2400,
    },
    "teclado musical": {
        "peso": 3.5,
        "valor": 3000,
    },
    "bicicleta": {
        "peso": 16,
        "valor": 1000,
    },
    "lições em dia": {
        "peso": 8,
        "valor": 5000,
    },
    "energético": {
        "peso": 2,
        "valor": 1500,
    },
    "docinhos para o stress": {
        "peso": 5,
        "valor": 3000,
    },
    "pente do daniel": {
        "peso": 1,
        "valor": 1000,
    },
    "cigarro do fazzio": {
        "peso": 2,
        "valor": 2000,
    },
    "grafeno da valeria": {
        "peso": 1,
        "valor": 3000,
    },
    "dopamina da juliana": {
        "peso": 5,
        "valor": 4000,
    },
    "nelson studart: o calango do nordeste": {
        "peso": 7,
        "valor": 7000,
    },
    "cortez (quem?)": {
        "peso": 6,
        "valor": 5000,
    },
    "chuteira do hernandes": {
        "peso": 3,
        "valor": 3500,
    },
    
}
NUM_OBJETOS = len(OBJETOS)
ORDEM_DOS_NOMES = list(sorted(OBJETOS.keys()))

In [7]:
# Busca por algoritmo genético

populacao = cria_populacao_inicial(TAMANHO_POP, NUM_OBJETOS)

# variaveis para o hall da fama
melhor_fitness_ja_visto = -float("inf")
melhor_individuo_ja_visto = [0] * NUM_OBJETOS

inicio = time.time()

for n in range(NUM_GERACOES):

    # Seleção
    fitness = funcao_objetivo_pop(populacao)
    populacao = funcao_selecao(populacao, fitness)

    # Cruzamento
    pais = populacao[0::2]
    maes = populacao[1::2]

    contador = 0

    for pai, mae in zip(pais, maes):
        if random.random() <= CHANCE_CRUZAMENTO:
            filho1, filho2 = funcao_cruzamento(pai, mae)
            populacao[contador] = filho1
            populacao[contador + 1] = filho2

        contador = contador + 2

    # Mutação
    for n in range(len(populacao)):
        if random.random() <= CHANCE_MUTACAO:
            individuo = populacao[n]
            populacao[n] = funcao_mutacao(individuo)

    # melhor individuo já visto até agora (hall da fama)
    fitness = funcao_objetivo_pop(populacao)
    maior_fitness = max(fitness)
    posicao = fitness.index(maior_fitness)
    individuo = populacao[posicao].copy()
    valor, peso = computa_mochila(individuo, OBJETOS, ORDEM_DOS_NOMES)
    if maior_fitness > melhor_fitness_ja_visto and peso <= LIMITE_DE_PESO:
        melhor_fitness_ja_visto = maior_fitness
        melhor_individuo_ja_visto = individuo
        print(f"Maior valor: {valor} | Peso: {peso}")

fim = time.time()

# reportando o melhor individuo encontrado
print()
print("Você deve pegar os seguintes itens:")
for pega_ou_nao, item in zip(melhor_individuo_ja_visto, ORDEM_DOS_NOMES):
    if pega_ou_nao == 1:
        print("+", item)
print()
valor_total, peso_total = computa_mochila(
    melhor_individuo_ja_visto, OBJETOS, ORDEM_DOS_NOMES
)
print(
    f"Com isso, sua mochila terá o valor de {valor_total} dinheiros "
    f"e peso de {peso_total} unidades de massa."
)

print(f'O código levou {round(fim-inicio,4)} segundos para ser executado')

Maior valor: 10150 | Peso: 15
Maior valor: 17000 | Peso: 15

Você deve pegar os seguintes itens:
+ Quadrinho super raro do Aranha-Homem da vida real
+ docinhos para o stress
+ nelson studart: o calango do nordeste

Com isso, sua mochila terá o valor de 17000 dinheiros e peso de 15 unidades de massa.
O código levou 0.006 segundos para ser executado


In [8]:
melhor_valor, melhor_peso = float('-inf'), float('inf')

inicio = time.time()

for individuo in itertools.product([0,1], repeat=len(OBJETOS.keys())):
    valor,peso = computa_mochila(individuo,OBJETOS,ORDEM_DOS_NOMES)
    
    if peso <= LIMITE_DE_PESO:
        if valor > melhor_valor:
            melhor_valor = valor
            melhor_peso = peso

fim = time.time()

print(f"A melhor mochila possível tem {melhor_valor} dinheiros e peso de {melhor_peso} unidades de massa.")
print(f'O código levou {round(fim-inicio,4)} segundos para ser executado')

A melhor mochila possível tem 22900 dinheiros e peso de 14.6 unidades de massa.
O código levou 0.1842 segundos para ser executado


#

## Conclusão

O último experimento testado na parte de algoritmos genéticos configura-se como um dos problemas resolvidos na matéria de Lógica Computacional, que é o problema das mochilas. Sendo classificado como um problema do tipo não-polinomial difícil, é necessário encontrar a mochila que permita levar os itens mais caros possíveis de acordo com uma restrição de peso imposta.

Utilizando uma seleção de roleta máxima e um cruzamento de ponto simples, o problema foi resolvido por meio da criação de indivíduos que são iguais aos da caixa binária. Dessa forma, cada indivíduo é uma lista com o tamanho do dicionário de objetos, onde o 0 representa a ausência do objeto e 1 a sua presença.

Assim como nos demais algoritmos genéticos aplicados nos experimentos do repositório, lidamos com abordagens probabilísticas e não determinísticas, não nos permitindo, portanto, concluir que a melhor escolha foi feita. Sendo assim, como forma de verificar os resultados, foi testado também a utilização de uma busca em grade.

Em alguns testes, o algoritmo genético encontrou a melhor solução, assim como a busca em grade, porém em um tempo maior do que a busca em grade, algo inesperado. A hipótese levantada para este resultado foi a de uma baixa quantidade de busca necessária dos objetos. Foi então criado um outro dicionário com uma quantidade maior de itens para verificação, de forma que a busca levasse um maior tempo para execução.

Em suma, é possível concluir que, para uma baixa quantidade de valores do dicionário, a busca em grade foi suficiente, demandando recursos similares para execução. Por outro lado, com um aumento da quantidade de valores, o algoritmo genético, apesar de não encontrar a melhor solução, foi mais eficiente em questão de tempo.

## Playground

